# First look at H2C Commissioning Data
This notebook will be run daily to give us a glance at the early commissioning data as it arrives at NRAO. The goal is to run some of our analysis steps and provide useful plots.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import numpy as np
import hera_qm as hqm
import hera_cal as hc
from pyuvdata import UVCal, UVData, UVFITS
import os
import sys
import glob
import uvtools as uvt
from astropy.time import Time

## Set up Env variables and get files

In [ ]:
# get data location
# Uncomment this line and delete the next when ready for the real deal
# data_path = os.environ['DATA_PATH']
data_path = '/users/abeardsl/lustre/2458042/'  # Subset of H1C IDR1 raw data files

# get xx and yy files
xxfiles = sorted(glob.glob("{0}/zen.*.*.xx.HH.uvh5".format(data_path)))
yyfiles = sorted(glob.glob("{0}/zen.*.*.yy.HH.uvh5".format(data_path)))
Nfiles = len(xxfiles)

xxfile_bases = map(os.path.basename, xxfiles)
yyfile_bases = map(os.path.basename, yyfiles)

xxfile_times = np.array(map(lambda x: '.'.join(os.path.basename(x).split('.')[1:3]), xxfiles), np.float)
yyfile_times = np.array(map(lambda y: '.'.join(os.path.basename(y).split('.')[1:3]), yyfiles), np.float)

# choose one for single-file plots
file_index = np.min([len(xxfiles)-1, 20])
xxfile = xxfiles[file_index]
yyfile = yyfiles[file_index]
xxfile_base = xxfile_bases[file_index]
yyfile_base = yyfile_bases[file_index]
file_jd = xxfile_times[file_index]

In [ ]:
# Load data
uvd_xx = UVData()
uvd_xx.read(xxfile)
uvd_xx.ants = np.unique(np.concatenate([uvd_xx.ant_1_array, uvd_xx.ant_2_array]))
uvd_yy = UVData()
uvd_yy.read(yyfile)
uvd_yy.ants = np.unique(np.concatenate([uvd_yy.ant_1_array, uvd_yy.ant_2_array]))

# Get metadata
freqs = uvd_xx.freq_array.squeeze() / 1e6
times = uvd_xx.time_array.reshape(uvd_xx.Ntimes, uvd_xx.Nbls)[:, 0]
jd_start = np.floor(times.min())
Nfreqs = len(freqs)
Ntimes = len(times)

# get redundant info
aa = hc.utils.get_aa_from_uv(uvd_xx)
info = hc.omni.aa_to_info(aa)
red_bls = np.array(info.get_reds())
antpos, ants = uvd_xx.get_ENU_antpos(pick_data_ants=False)
ants = sorted(ants)
ants_red = sorted(np.unique(np.concatenate(red_bls)))
Nants = len(ants)
Nside = int(np.ceil(np.sqrt(Nants)))
Yside = int(np.ceil(float(Nants)/Nside))

## Plot Autos

In [ ]:
### plot autos
t_index = 0
jd = times[t_index]
utc = Time(jd, format='jd').datetime

xlim = (-50, Nfreqs+50)
ylim = (-10,30)

fig, axes = plt.subplots(Yside, Nside, figsize=(14,14), dpi=75)
fig.subplots_adjust(wspace=0.2, hspace=0.2)
fig.suptitle("JD = {0}, time = {1} UTC".format(jd, utc), fontsize=14)
fig.tight_layout(rect=(0, 0, 1, 0.95))

k = 0
for i in range(Yside):
    for j in range(Nside):
        ax = axes[i,j]
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        if k < Nants:
            px, = ax.plot(10*np.log10(np.abs(uvd_xx.get_data((ants[k], ants[k]))[t_index])),
                          color='steelblue', alpha=0.75, linewidth=3)
            py, = ax.plot(10*np.log10(np.abs(uvd_yy.get_data((ants[k], ants[k]))[t_index])),
                          color='darkorange', alpha=0.75, linewidth=3)
            ax.grid(True, which='both')
            ax.set_title(str(ants[k]), fontsize=14)
            if k == 0:
                ax.legend([px, py], ['East', 'North'], fontsize=12)
        else:
            ax.axis('off')
        if j != 0:
            ax.set_yticklabels([])
        else:
            [t.set_fontsize(12) for t in ax.get_yticklabels()]
            ax.set_ylabel(r'$10\cdot\log_{10}$ amplitude', fontsize=14)
        if i != Yside-1:
            ax.set_xticklabels([])
        else:
            [t.set_fontsize(12) for t in ax.get_xticklabels()]
            ax.set_xlabel('freq channel', fontsize=14)

        k += 1

## Plot auto amplitude at a specific frequency over entire night


In [ ]:
def timeseries_data(files, nu_index=500):
    print "gather nightly data for JD = {0}".format(jd_start)
    uv = UVData()
    uv.read(files, ant_str='auto', freq_chans=[nu_index])
    amps = np.abs(uv.data_array[:, 0, 0, 0].reshape(uv.Ntimes, uv.Nants_data))
    times = np.unique(uv.time_array)
    return np.array(amps).T, np.array(times)

nu_index = 500
amps_timeseries_xx, times_xx = timeseries_data(xxfiles, nu_index=nu_index)
amps_timeseries_yy, times_yy = timeseries_data(yyfiles, nu_index=nu_index)
times_xx = times_xx % 1
times_yy = times_yy % 1

In [ ]:
# plot autos
ylim = (0, np.median(np.abs(amps_timeseries_xx))*3)
ylim = (1e-1, np.abs(amps_timeseries_xx).max() * 3)

fig, axes = plt.subplots(Yside, Nside, figsize=(14,14), dpi=75)
fig.subplots_adjust(wspace=0.2, hspace=0.2)
fig.suptitle("autocorrelation from {0} -- {1} at freq bin {2}".format(times[0], times[-1], nu_index), fontsize=14)
fig.tight_layout(rect=(0, 0, 1, 0.95))

k = 0
for i in range(Yside):
    for j in range(Nside):
        ax = axes[i,j]
        ax.set_ylim(ylim)
        if k < Nants:
            ax.set_yscale('log')
            px, = ax.plot(times_xx, np.abs(amps_timeseries_xx[k]), color='steelblue', alpha=0.5, linewidth=3)
            py, = ax.plot(times_yy, np.abs(amps_timeseries_yy[k]), color='darkorange', alpha=0.5, linewidth=3)
            ax.grid(True)
            ax.set_title(str(ants[k]), fontsize=14)
            if k == 0:
                ax.legend([px, py], ['East', 'North'], fontsize=12)
        else:
            ax.axis('off')
        if j != 0:
            ax.set_yticklabels([])
        else:
            [t.set_fontsize(12) for t in ax.get_yticklabels()]
            ax.set_ylabel('amplitude', fontsize=14)
        if i != Yside-1:
            ax.set_xticklabels([])
        else:
            [t.set_fontsize(12) for t in ax.get_xticklabels()]
            [t.set_rotation(25) for t in ax.get_xticklabels()]
            ax.xaxis.set_major_formatter(FormatStrFormatter('%.3f'))
            ax.set_xlabel('decimal of JD', fontsize=14)
        k += 1

## Run ant metrics

In [ ]:
# Run all the metrics cause why not
# Output metrics files will look like: zen.JD.JD.HH.uvh5.ant_metrics.hdf5
ant_metrics_run(xxfiles, pols=['xx', 'yy', 'xy', 'xy'], crossCut=5.0,
                deadCut=5.0, alwaysDeadCut=10.0, metrics_path='',
                extension='.ant_metrics.hdf5', vis_format='h5',
                verbose=False, history='',
                run_mean_vij=True, run_red_corr=True,
                run_cross_pols=True)